
# Projet Maison 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd
import math
import json
# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
GEO = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")

# A COMPLETER avec les colonnes
# - lat, lon : latitude et longitude des communes
GEO['lat'] = GEO['geo_point_2d'].str.split(',').str.get(0)
GEO['lon'] = GEO['geo_point_2d'].str.split(',').str.get(1)

GEO = GEO.astype({'lat':float, 'lon':float})

# - cp_ville : Code Postal + " " + "Commune"
GEO['cp_ville'] = GEO['Code Postal'] + " " + GEO['Commune']

**Partie A**

- Compléter le chargement des données en ajoutant au dataframe `GEO`
    - les colonnes "lat" et "lon" avec la latitude et la longitude des communes
    - une colonne "cp_ville" avec le Code Postal + un espace + et le nom de la Commune
- Ecrire une fonction `search_city(lat, lon)` qui retourne le "cp_ville" de la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ecrire une fonction `dms2dec(deg, min, sec)` qui convertit les degrés, minutes, secondes en valeur numérique pour pouvoir utiliser la fonction précédente avec un GPS.

In [3]:
# fonction recherche de ville
def search_city(lat, long):
    sub_geo = pd.DataFrame(GEO[['cp_ville', 'lat', 'lon']])
    sub_geo['distance'] = np.sqrt((lat - sub_geo['lat'])**2 + (long - sub_geo['lon'])**2)
    return sub_geo.loc[sub_geo['distance'] == sub_geo['distance'].min()]['cp_ville'].values

In [4]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + (mn / 60) + (sec / 3600)

In [5]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = np.random.uniform(a, b)
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
lon = np.random.uniform(a, b)

search_city(lat, lon)

array(['71330 BOSJEAN'], dtype=object)

In [6]:
# à partir de coordonnées GPS précises
search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

array(['91120 PALAISEAU'], dtype=object)

**Partie B**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python **json** pour parser les valeurs de la colonne "geo_shape" et mettre le résultat (`Series`) dans la variable `GEO_SHAPE`.
- Ecrire une fonction `get_types()` qui retourne le décompte (`value_counts()`) des valeurs accédées avec la clé "type".
- Ecrire une fonction `get_coordinates_len()` qui retourne le décompte (`value_counts()`) des longueurs des listes accédées avec la clé "coordinates".
- Ecrire une fonction `get_most_complex_city()` qui retourne la commune est constituée du plus grand nombre de polygones ?
- Ecrire une fonction `get_nb_cities_2_polygons()` qui retourne  le nombre de villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- **Facultatif :**
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : on pourra installer la librairie **shapely**, utiliser la classe Polygon de **shapely.geometry**  et la méthode `contains()`. Sur Windows **shapely** peut nécessiter d'installer manuellement la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [7]:
# GEO_SHAPE
# La variable GEO_SHAPE doit contenir une Serie
# correspondant aux valeurs de la colonne "geo_shape" parsées avec la librairie json
data = [dict] * len(GEO)
for i in range(len(GEO)):
    data[i] = json.loads(GEO['geo_shape'][i])
    
GEO_SHAPE = pd.Series(data = data, dtype=object)

In [8]:
# value_counts des valeurs "type"
def get_types():
    # returns dictionary of types and value_counts() of types -- me
    dict_types = {}
    for i in range(len(GEO_SHAPE)):
        type_i = GEO_SHAPE[i]['type']
        
        if type_i in dict_types:
            dict_types[type_i] = dict_types[type_i] + 1
        else:
            dict_types[type_i] = 1
    return dict_types

In [9]:
# value_counts des longueurs de "coordinates"
def get_coordinates_len():
    # returns dictionary of list_length (a.k.a number of polygons) and value_counts() of each list length -- me
    dict_coordinates = {}
    
    for i in range(len(GEO_SHAPE)):
        coordinates_i = GEO_SHAPE[i]['coordinates']
        len_coordinates_i = len(coordinates_i)
        if len_coordinates_i in dict_coordinates:
            dict_coordinates[len_coordinates_i] = dict_coordinates[len_coordinates_i] + 1
        else:
            dict_coordinates[len_coordinates_i] = 1
            
    return dict_coordinates

In [10]:
# commune constituée du plus grand nombre de polygones
def get_most_complex_city():
    
    polygon_size = [int] * len(GEO_SHAPE)
    for i in range(len(GEO_SHAPE)):
        polygon_size[i] = len(GEO_SHAPE[i]['coordinates'])
        
    GEO['polygon_size'] = polygon_size

    return GEO.loc[GEO['polygon_size']==GEO['polygon_size'].max()]['cp_ville'].values[0]

In [11]:
# nombre de communes constituées 2 polygones mais de type Polygon
def get_nb_cities_2_polygons():
    
    geo_shape_type = [str] * len(GEO_SHAPE)
    for i in range(len(GEO_SHAPE)):
        geo_shape_type[i] = GEO_SHAPE[i]['type']
        
    GEO['geo_shape_type'] = geo_shape_type
    
    return len(GEO.loc[(GEO['geo_shape_type'] == 'Polygon') & (GEO['polygon_size']==2)])

In [12]:
# tests
import unittest

class Session2Test(unittest.TestCase):
    
    def test_partie_A1(self):
        # on applique la fonction cherche_ville() à une coordonnée tirée au hasard
        np.random.seed(0)
        a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
        lat = np.random.uniform(a, b)
        a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
        lon = np.random.uniform(a, b)

        cp_ville = search_city(lat, lon)
        self.assertEqual(cp_ville, "71330 BOSJEAN")
        
    def test_partie_A2(self):
        # à partir de coordonnées GPS précises
        cp_ville = search_city(dms2dec(48, 42, 52), dms2dec(2, 14, 45))
        self.assertEqual(cp_ville, "91120 PALAISEAU")
        
    def test_partie_B1(self):
        # check types
        dico = get_types()
        self.assertEqual(dico["Polygon"], 36670)
        self.assertEqual(dico["MultiPolygon"], 72)
        
    def test_partie_B2(self):
        # check coordinates len
        dico = get_coordinates_len()
        self.assertEqual(dico[1], 36660)
        self.assertEqual(dico[2], 80)

    def test_partie_B3(self):
        # check most complex city
        cp_ville = get_most_complex_city()
        self.assertEqual(cp_ville, "83400 HYERES")
        
    def test_partie_B4(self):
        # check nb cities 2 polygons
        nb = get_nb_cities_2_polygons()
        self.assertEqual(nb, 10)

In [13]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session2Test)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_partie_A1 (__main__.Session2Test) ... ok
test_partie_A2 (__main__.Session2Test) ... ok
test_partie_B1 (__main__.Session2Test) ... ok
test_partie_B2 (__main__.Session2Test) ... ok
test_partie_B3 (__main__.Session2Test) ... ok
test_partie_B4 (__main__.Session2Test) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.528s

OK
